In [ ]:
# ИНСТРУКЦИЯ ПО ИСПОЛЬЗОВАНИЮ:
#ПОСЛЕДОВАТЕЛЬНОСТЬ ЗАПУСКА:

#СКРИПТ 1 → Запустить ОДИН РАЗ в начале работы
#Устанавливает зависимости
#Загружает модель в память

#СКРИПТ 2 → Запускать при каждом новом файле
#Загружает Excel файл через интерфейс
#Извлекает тексты

#Выберите один из вариантов:
#СКРИПТ 3 → Обработать ВСЕ тексты сразу
#СКРИПТ 4 → Обработать ОДИН текст (для выборочной обработки)

#СКРИПТ 5 → Сохранить результаты
#Когда все тексты обработаны

#ВАЖНЫЕ ЗАМЕЧАНИЯ:
#Модель в памяти: После запуска Скрипта 1 модель остается в памяти до перезагрузки Colab
#Промежуточное сохранение: Каждый текст сохраняется в отдельный CSV файл сразу после обработки
#Можно прерывать: Если обработка прервалась, можно запустить снова - уже обработанные тексты пропускаются
#Память GPU: Если не хватает памяти, в Скрипте 1 можно поменять модель на более легкую (Qwen1.5-0.5B → gpt2)

In [10]:
# script1_setup.py
"""
СКРИПТ 1: УСТАНОВКА ЗАВИСИМОСТЕЙ И ЗАГРУЗКА МОДЕЛИ
ЗАПУСТИТЬ ТОЛЬКО ОДИН РАЗ В НАЧАЛЕ РАБОТЫ!
"""

print("="*60)
print("СКРИПТ 1: УСТАНОВКА И ЗАГРУЗКА МОДЕЛИ")
print("ЗАПУСТИТЕ ЭТОТ СКРИПТ ТОЛЬКО ОДИН РАЗ!")
print("="*60)

# Установка библиотек
!pip install -q transformers accelerate torch pandas numpy tqdm openpyxl

import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
import math
import gc
import os
import sys
from tqdm import tqdm

# Глобальные переменные для модели
GLOBAL_TOKENIZER = None
GLOBAL_MODEL = None
GLOBAL_DEVICE = None
def load_global_model(model_name="Qwen/Qwen1.5-0.5B"):

    """
    Загружает модель в глобальные переменные
    Qwen1.5-0.5B - достаточно легкая для Colab
    """
    global GLOBAL_TOKENIZER, GLOBAL_MODEL, GLOBAL_DEVICE

    print(f"🤖 Загрузка модели: {model_name}")

    # Токенизатор
    GLOBAL_TOKENIZER = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    GLOBAL_TOKENIZER.pad_token = GLOBAL_TOKENIZER.eos_token

    # Определяем устройство
    GLOBAL_DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"📟 Устройство: {GLOBAL_DEVICE}")

    # Модель
    try:
        GLOBAL_MODEL = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16 if GLOBAL_DEVICE == "cuda" else torch.float32,
            low_cpu_mem_usage=True,
            device_map="auto" if GLOBAL_DEVICE == "cuda" else None
        )
        GLOBAL_MODEL.eval()
        print("✅ Модель успешно загружена в глобальные переменные")

    except Exception as e:
        print(f"❌ Ошибка загрузки модели: {e}")
        print("🔄 Пробуем альтернативную модель...")

        # Альтернатива - еще более легкая модель
        model_name = "gpt2"  # Самая легкая, но на русском работает хуже
        GLOBAL_TOKENIZER = AutoTokenizer.from_pretrained(model_name)
        GLOBAL_TOKENIZER.pad_token = GLOBAL_TOKENIZER.eos_token

        GLOBAL_MODEL = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if GLOBAL_DEVICE == "cuda" else torch.float32
        )
        GLOBAL_MODEL.eval()
        print("✅ Альтернативная модель загружена")

    # Информация о памяти
    if GLOBAL_DEVICE == "cuda":
        print(f"💾 Память GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

    return GLOBAL_TOKENIZER, GLOBAL_MODEL, GLOBAL_DEVICE

# Загружаем модель сразу
GLOBAL_TOKENIZER, GLOBAL_MODEL, GLOBAL_DEVICE = load_global_model()

print("\n" + "="*60)
print("МОДЕЛЬ ГОТОВА К РАБОТЕ!")
print("Теперь запустите СКРИПТ 2 для загрузки файла")
print("="*60)

СКРИПТ 1: УСТАНОВКА И ЗАГРУЗКА МОДЕЛИ
ЗАПУСТИТЕ ЭТОТ СКРИПТ ТОЛЬКО ОДИН РАЗ!
🤖 Загрузка модели: Qwen/Qwen1.5-0.5B
📟 Устройство: cpu


Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


✅ Модель успешно загружена в глобальные переменные

МОДЕЛЬ ГОТОВА К РАБОТЕ!
Теперь запустите СКРИПТ 2 для загрузки файла


In [11]:
# script2_load_excel.py - ИСПРАВЛЕННАЯ ВЕРСИЯ
"""
СКРИПТ 2: ЗАГРУЗКА EXCEL-ФАЙЛА И ИЗВЛЕЧЕНИЕ ТЕКСТОВ
ИСПРАВЛЕННАЯ ВЕРСИЯ - ЧИТАЕТ ВСЕ ТЕКСТЫ
"""

print("="*60)
print("СКРИПТ 2: ЗАГРУЗКА EXCEL-ФАЙЛА")
print("="*60)

from google.colab import files
import pandas as pd
import os

# Глобальная переменная для текстов
GLOBAL_TEXTS = {}

def upload_excel_file():
    """Загружает Excel файл через интерфейс Colab"""
    print("📤 ЗАГРУЗИТЕ EXCEL-ФАЙЛ С ТЕКСТАМИ")
    print("Ожидание загрузки файла...")

    uploaded = files.upload()

    if not uploaded:
        print("❌ Файл не загружен")
        return None

    # Сохраняем первый загруженный файл
    file_name = list(uploaded.keys())[0]

    # Сохраняем файл для дальнейшего использования
    with open(file_name, 'wb') as f:
        f.write(uploaded[file_name])

    print(f"✅ Файл сохранен: {file_name}")
    print(f"📏 Размер: {len(uploaded[file_name]) / 1024:.1f} KB")

    return file_name

def extract_texts_from_excel(excel_path):
    """
    ИСПРАВЛЕННАЯ ФУНКЦИЯ: Извлекает тексты из Excel файла
    Правильно обрабатывает структуру KeyList
    """
    print(f"\n📖 Чтение Excel файла: {excel_path}")

    try:
        # Читаем лист KeyList
        key_df = pd.read_excel(excel_path, sheet_name='KeyList')

        # Отладочная информация
        print("\n🔍 СОДЕРЖИМОЕ KeyList:")
        print(key_df)
        print(f"\nКолонки KeyList: {list(key_df.columns)}")
        print(f"Строк в KeyList: {len(key_df)}")

        texts = {}
        print("\n📋 ПОИСК ТЕКСТОВ:")

        # ВАЖНО: Ваша структура имеет заголовок в первой строке
        # и данные начинаются со второй строки

        # Способ 1: Ищем все строки, где textSheet начинается с 'Text_'
        for idx, row in key_df.iterrows():
            try:
                # Проверяем, что это строка с данными (не заголовок)
                text_sheet = str(row['textSheet']).strip()

                # Пропускаем заголовок
                if text_sheet.lower() == 'textsheet' or text_sheet == 'nan' or pd.isna(row['textSheet']):
                    print(f"  Пропускаем заголовок: строка {idx}")
                    continue

                # Извлекаем данные
                text_id = str(row['textSheet']).strip()
                title = str(row['Titles']).strip()

                print(f"  🔹 Найден: {text_id} - '{title}' (строка {idx})")

                # Читаем лист с текстом
                try:
                    df = pd.read_excel(excel_path, sheet_name=text_id)

                    # Правильное извлечение текста
                    text_parts = []

                    # Проходим по всем столбцам (A-J)
                    for col in df.columns:
                        # Берем все не-NaN значения из столбца
                        column_words = df[col].dropna().astype(str).tolist()
                        # Добавляем слова
                        text_parts.extend(column_words)

                    # Объединяем ВСЕ слова в один текст
                    full_text = ' '.join(text_parts)

                    # Убираем лишние пробелы
                    full_text = ' '.join(full_text.split())

                    texts[text_id] = {
                        'title': title,
                        'text': full_text,
                        'words_count': len(text_parts),
                        'chars_count': len(full_text)
                    }

                    print(f"     ✓ Успешно: {len(text_parts)} слов, {len(full_text)} символов")

                except Exception as sheet_error:
                    print(f"     ✗ Ошибка чтения листа '{text_id}': {sheet_error}")
                    texts[text_id] = {
                        'title': title,
                        'text': '',
                        'words_count': 0,
                        'chars_count': 0,
                        'error': str(sheet_error)
                    }

            except Exception as row_error:
                print(f"  ✗ Ошибка в строке {idx}: {row_error}")
                continue

        # Альтернативный способ, если первый не сработал
        if not texts:
            print("\n⚠️  Первый способ не сработал, пробуем альтернативный...")

            # Пробуем найти все листы, которые начинаются с 'Text_'
            xls = pd.ExcelFile(excel_path)
            sheet_names = xls.sheet_names

            print(f"Все листы в файле: {sheet_names}")

            for sheet in sheet_names:
                if sheet.startswith('Text_'):
                    try:
                        # Пытаемся найти заголовок в KeyList
                        title = f"Неизвестный текст ({sheet})"

                        # Ищем заголовок в KeyList
                        for idx, row in key_df.iterrows():
                            if str(row['textSheet']).strip() == sheet:
                                title = str(row['Titles']).strip()
                                break

                        df = pd.read_excel(excel_path, sheet_name=sheet)

                        # Извлекаем текст
                        text_parts = []
                        for col in df.columns:
                            column_words = df[col].dropna().astype(str).tolist()
                            text_parts.extend(column_words)

                        full_text = ' '.join(text_parts)
                        full_text = ' '.join(full_text.split())

                        texts[sheet] = {
                            'title': title,
                            'text': full_text,
                            'words_count': len(text_parts),
                            'chars_count': len(full_text)
                        }

                        print(f"  🔹 {sheet}: '{title}' - {len(text_parts)} слов")

                    except Exception as e:
                        print(f"  ✗ Ошибка листа {sheet}: {e}")

        print(f"\n✅ ИТОГО ИЗВЛЕЧЕНО: {len(texts)} текстов")

        # Показываем все найденные тексты
        if texts:
            print("\n📄 НАЙДЕННЫЕ ТЕКСТЫ:")
            for text_id, info in texts.items():
                print(f"  {text_id}: {info['title']}")
                if info['text']:
                    sample = info['text'][:100] + "..." if len(info['text']) > 100 else info['text']
                    print(f"     '{sample}'")
                print()

        return texts

    except Exception as e:
        print(f"❌ КРИТИЧЕСКАЯ ОШИБКА ЧТЕНИЯ ФАЙЛА: {e}")
        import traceback
        traceback.print_exc()
        return {}

def load_texts_interactive():
    """
    Интерактивная загрузка текстов из Excel
    """
    global GLOBAL_TEXTS

    # 1. Загружаем файл
    excel_file = upload_excel_file()
    if not excel_file:
        return None

    # 2. Извлекаем тексты
    GLOBAL_TEXTS = extract_texts_from_excel(excel_file)

    if not GLOBAL_TEXTS:
        print("❌ Не удалось извлечь тексты из файла")
        return None

    print(f"\n🎯 ГОТОВО К ОБРАБОТКЕ: {len(GLOBAL_TEXTS)} текстов")

    # Показываем меню
    print("\n" + "="*60)
    print("ВЫ МОЖЕТЕ:")
    print("1. Обработать ВСЕ тексты сразу (запустите СКРИПТ 3)")
    print("2. Обработать ПО ОДНОМУ тексту (запустите СКРИПТ 4)")
    print("="*60)

    return GLOBAL_TEXTS

# Запускаем загрузку
GLOBAL_TEXTS = load_texts_interactive()

# Сохраняем информацию о загруженных текстах для отладки
if GLOBAL_TEXTS:
    with open('loaded_texts_info.txt', 'w', encoding='utf-8') as f:
        f.write(f"Загружено текстов: {len(GLOBAL_TEXTS)}\n")
        f.write("="*50 + "\n")
        for text_id, info in GLOBAL_TEXTS.items():
            f.write(f"{text_id}: {info['title']}\n")
            f.write(f"Слов: {info['words_count']}, Символов: {info['chars_count']}\n")
            f.write(f"Первые 200 символов: {info['text'][:200]}...\n")
            f.write("-"*50 + "\n")

    print("\n📝 Информация о загруженных текстах сохранена в 'loaded_texts_info.txt'")

СКРИПТ 2: ЗАГРУЗКА EXCEL-ФАЙЛА
📤 ЗАГРУЗИТЕ EXCEL-ФАЙЛ С ТЕКСТАМИ
Ожидание загрузки файла...


Saving data_exp_texts_14.xlsx to data_exp_texts_14 (2).xlsx
✅ Файл сохранен: data_exp_texts_14 (2).xlsx
📏 Размер: 11.3 KB

📖 Чтение Excel файла: data_exp_texts_14 (2).xlsx

🔍 СОДЕРЖИМОЕ KeyList:
  textSheet           Titles
0   Text_14  Азиатский стиль

Колонки KeyList: ['textSheet', 'Titles']
Строк в KeyList: 1

📋 ПОИСК ТЕКСТОВ:
  🔹 Найден: Text_14 - 'Азиатский стиль' (строка 0)
     ✓ Успешно: 137 слов, 1179 символов

✅ ИТОГО ИЗВЛЕЧЕНО: 1 текстов

📄 НАЙДЕННЫЕ ТЕКСТЫ:
  Text_14: Азиатский стиль
     'В 2022 году южнокорейским концерном Самсунг публике было представлено новое поколение бытовой техник...'


🎯 ГОТОВО К ОБРАБОТКЕ: 1 текстов

ВЫ МОЖЕТЕ:
1. Обработать ВСЕ тексты сразу (запустите СКРИПТ 3)
2. Обработать ПО ОДНОМУ тексту (запустите СКРИПТ 4)

📝 Информация о загруженных текстах сохранена в 'loaded_texts_info.txt'


In [ ]:
# script3_process_all.py
"""
СКРИПТ 3: ОБРАБОТКА ВСЕХ ТЕКСТОВ СРАЗУ
ЗАПУСКАТЬ ПОСЛЕ СКРИПТА 1 и 2
"""

print("="*60)
print("СКРИПТ 3: ОБРАБОТКА ВСЕХ ТЕКСТОВ")
print("="*60)

# Проверяем, что модель загружена
if 'GLOBAL_TOKENIZER' not in globals():
    print("❌ ОШИБКА: Модель не загружена!")
    print("   Сначала запустите СКРИПТ 1")
    raise SystemExit

# Проверяем, что тексты загружены
if 'GLOBAL_TEXTS' not in globals() or not GLOBAL_TEXTS:
    print("❌ ОШИБКА: Тексты не загружены!")
    print("   Сначала запустите СКРИПТ 2")
    raise SystemExit

print(f"📊 БУДЕТ ОБРАБОТАНО: {len(GLOBAL_TEXTS)} текстов")

# Функция обработки одного текста (ТОЧНО КАК В ИСХОДНОМ КОДЕ)
def process_single_text_correct(text, text_id, max_tokens=512):
    """
    Обработка одного текста - ТОЧНО КАК В ИСХОДНОМ КОДЕ
    с правильной токенизацией через модель
    """
    if not text or not text.strip():
        print(f"   Текст {text_id} пустой, пропускаем...")
        return None

    print(f"\n   🔹 Начало обработки: {text_id}")
    print(f"      Символов: {len(text)}")

    try:
        # ТОКЕНИЗАЦИЯ ЧЕРЕЗ МОДЕЛЬ (правильно!)
        inputs = GLOBAL_TOKENIZER(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=max_tokens
        )
        input_ids = inputs.input_ids.to(GLOBAL_DEVICE)
        seq_len = input_ids.shape[1]

        print(f"      Токенов (по модели): {seq_len}")

        results = []

        # ПЕРВЫЙ ТОКЕН НЕ ИМЕЕТ КОНТЕКСТА → ПРОПУСКАЕМ
        with torch.no_grad():
            for i in range(1, seq_len):
                # БЕРЕМ ПРЕФИКС
                prefix = input_ids[:, :i]

                # ПРОГОН ЧЕРЕЗ МОДЕЛЬ
                outputs = GLOBAL_MODEL(prefix)
                logits = outputs.logits

                # ВЕРОЯТНОСТИ ДЛЯ СЛЕДУЮЩЕГО ТОКЕНА
                next_token_logits = logits[0, -1, :]
                next_token_probs = torch.softmax(next_token_logits, dim=-1)

                # ИСТИННЫЙ ТОКЕН
                target_id = input_ids[0, i].item()
                prob = next_token_probs[target_id].item()

                # ДЕКОДИРУЕМ ТОКЕН
                token_str = GLOBAL_TOKENIZER.decode(
                    [target_id],
                    skip_special_tokens=True
                ).strip()

                results.append({
                    "position": i,
                    "token": token_str,
                    "token_id": target_id,
                    "probability": prob,
                    "log_probability": math.log(prob) if prob > 0 else float('-inf')
                })

                # ОЧИСТКА ПАМЯТИ
                del outputs, logits, next_token_logits, next_token_probs
                if GLOBAL_DEVICE == "cpu":
                    gc.collect()

        print(f"      ✅ Обработано токенов: {len(results)}")
        return results

    except Exception as e:
        print(f"      ❌ Ошибка обработки: {str(e)}")
        return None

# Основная функция обработки
def process_all_texts():
    """Обрабатывает все загруженные тексты"""
    all_results = []
    processed_count = 0

    print("\n🚀 НАЧИНАЕМ ОБРАБОТКУ ВСЕХ ТЕКСТОВ")
    print("Это может занять время...")
    print("-" * 50)

    for text_id, text_info in GLOBAL_TEXTS.items():
        print(f"\n[{processed_count + 1}/{len(GLOBAL_TEXTS)}] {text_id}: {text_info['title']}")

        # Проверяем, не обработан ли уже
        output_file = f"{text_id}_probabilities.csv"
        if os.path.exists(output_file):
            print(f"   ⚠️  Файл уже существует, пропускаем...")
            try:
                df = pd.read_csv(output_file)
                all_results.append(df)
                processed_count += 1
                continue
            except:
                pass

        # Обработка текста
        results = process_single_text_correct(
            text=text_info['text'],
            text_id=text_id,
            max_tokens=512  # Для экономии времени и памяти
        )

        if results:
            # Сохраняем в DataFrame
            df = pd.DataFrame(results)
            df.insert(0, 'text_id', text_id)
            df.insert(1, 'title', text_info['title'])

            # Сохраняем в CSV
            df.to_csv(output_file, index=False, encoding='utf-8-sig')
            all_results.append(df)
            processed_count += 1

            print(f"   ✅ Сохранено в: {output_file}")
            print(f"      Средняя вероятность: {df['probability'].mean():.8f}")

        # Очистка памяти между текстами
        if GLOBAL_DEVICE == "cuda":
            torch.cuda.empty_cache()
        gc.collect()

    return all_results, processed_count

# Запускаем обработку
try:
    results, count = process_all_texts()

    print("\n" + "="*60)
    print(f"🎉 ОБРАБОТКА ЗАВЕРШЕНА!")
    print(f"✅ Обработано текстов: {count}/{len(GLOBAL_TEXTS)}")
    print(f"📁 Результаты сохранены в отдельных CSV файлах")
    print("="*60)

    # Создаем сводный отчет
    if results:
        summary_data = []
        for df in results:
            text_id = df['text_id'].iloc[0]
            summary_data.append({
                'text_id': text_id,
                'title': df['title'].iloc[0],
                'tokens_processed': len(df),
                'avg_probability': df['probability'].mean(),
                'min_probability': df['probability'].min(),
                'max_probability': df['probability'].max()
            })

        summary_df = pd.DataFrame(summary_data)
        summary_df.to_csv('ALL_TEXTS_SUMMARY.csv', index=False, encoding='utf-8-sig')

        print("\n📊 СВОДНЫЙ ОТЧЕТ:")
        print(summary_df.to_string())
        print(f"\n📁 Сводный отчет сохранен: ALL_TEXTS_SUMMARY.csv")

except KeyboardInterrupt:
    print("\n⚠️  ОБРАБОТКА ПРЕРВАНА ПОЛЬЗОВАТЕЛЕМ")
    print("   Частичные результаты сохранены")
except Exception as e:
    print(f"\n❌ ОШИБКА ПРИ ОБРАБОТКЕ: {e}")

СКРИПТ 3: ОБРАБОТКА ВСЕХ ТЕКСТОВ
📊 БУДЕТ ОБРАБОТАНО: 2 текстов

🚀 НАЧИНАЕМ ОБРАБОТКУ ВСЕХ ТЕКСТОВ
Это может занять время...
--------------------------------------------------

[1/2] Text_1: Оборотни без погон

   🔹 Начало обработки: Text_1
      Символов: 1109
      Токенов (по модели): 373
      ✅ Обработано токенов: 372
   ✅ Сохранено в: Text_1_probabilities.csv
      Средняя вероятность: 0.24421602

[2/2] Text_2: Призрак озеры

   🔹 Начало обработки: Text_2
      Символов: 1039
      Токенов (по модели): 407
      ✅ Обработано токенов: 406
   ✅ Сохранено в: Text_2_probabilities.csv
      Средняя вероятность: 0.24990150

🎉 ОБРАБОТКА ЗАВЕРШЕНА!
✅ Обработано текстов: 2/2
📁 Результаты сохранены в отдельных CSV файлах

📊 СВОДНЫЙ ОТЧЕТ:
  text_id               title  tokens_processed  avg_probability  min_probability  max_probability
0  Text_1  Оборотни без погон               372         0.244216     6.015322e-07         0.998478
1  Text_2       Призрак озеры               406         0.

In [13]:
# script4_process_single.py
"""
СКРИПТ 4: ОБРАБОТКА ОДНОГО ВЫБРАННОГО ТЕКСТА
Запускать если хотите обрабатывать по одному тексту
"""

print("="*60)
print("СКРИПТ 4: ОБРАБОТКА ОДНОГО ТЕКСТА")
print("="*60)

# Проверки
if 'GLOBAL_TEXTS' not in globals() or not GLOBAL_TEXTS:
    print("❌ Тексты не загружены! Запустите СКРИПТ 2")
    raise SystemExit

# Показываем доступные тексты
print("📋 ДОСТУПНЫЕ ТЕКСТЫ:")
for i, (text_id, info) in enumerate(GLOBAL_TEXTS.items(), 1):
    print(f"{i:2}. {text_id:10} - {info['title']:30} ({info['words_count']} слов)")

# Выбор текста
try:
    choice = int(input("\n🎯 Введите номер текста для обработки: ")) - 1

    text_ids = list(GLOBAL_TEXTS.keys())
    if 0 <= choice < len(text_ids):
        selected_id = text_ids[choice]
        selected_info = GLOBAL_TEXTS[selected_id]

        print(f"\n✅ ВЫБРАН: {selected_id} - {selected_info['title']}")
        print(f"   Символов: {selected_info['chars_count']}")

        if 'process_single_text_correct' not in globals():
            raise RuntimeError(
                "❌ Функция process_single_text_correct не найдена.\n"
                "Запусти сначала ячейку со СКРИПТОМ 3."
            )


        results = process_single_text_correct(
            text=selected_info['text'],
            text_id=selected_id,
            max_tokens=768  # Можно увеличить для более точной обработки
        )

        if results:
            df = pd.DataFrame(results)
            df.insert(0, 'text_id', selected_id)
            df.insert(1, 'title', selected_info['title'])

            output_file = f"{selected_id}_probabilities.csv"
            df.to_csv(output_file, index=False, encoding='utf-8-sig')

            print(f"\n📊 РЕЗУЛЬТАТЫ ОБРАБОТКИ:")
            print(f"   Файл: {output_file}")
            print(f"   Токенов: {len(df)}")
            print(f"   Средняя вероятность: {df['probability'].mean():.8f}")
            print(f"   Минимальная вероятность: {df['probability'].min():.8f}")
            print(f"   Максимальная вероятность: {df['probability'].max():.8f}")

            # Показываем первые 10 токенов
            print(f"\n📝 ПЕРВЫЕ 10 ТОКЕНОВ:")
            print(df[['position', 'token', 'probability']].head(10).to_string())

    else:
        print("❌ Неверный номер текста")

except ValueError:
    print("❌ Введите число!")
except Exception as e:
    print(f"❌ Ошибка: {e}")

СКРИПТ 4: ОБРАБОТКА ОДНОГО ТЕКСТА
📋 ДОСТУПНЫЕ ТЕКСТЫ:
 1. Text_14    - Азиатский стиль                (137 слов)

✅ ВЫБРАН: Text_14 - Азиатский стиль
   Символов: 1179

   🔹 Начало обработки: Text_14
      Символов: 1179
      Токенов (по модели): 402
      ✅ Обработано токенов: 401

📊 РЕЗУЛЬТАТЫ ОБРАБОТКИ:
   Файл: Text_14_probabilities.csv
   Токенов: 401
   Средняя вероятность: 0.32305052
   Минимальная вероятность: 0.00000056
   Максимальная вероятность: 0.99914443

📝 ПЕРВЫЕ 10 ТОКЕНОВ:
   position token  probability
0         1           0.003031
1         2     2     0.412016
2         3     0     0.941462
3         4     2     0.088743
4         5     2     0.138873
5         6  году     0.057012
6         7     �     0.001066
7         8     �     0.588818
8         9     ж     0.178416
9        10     н     0.085441


In [ ]:
# script5_save_results.py
"""
СКРИПТ 5: СОХРАНЕНИЕ РЕЗУЛЬТАТОВ НА GOOGLE DRIVE И СКАЧИВАНИЕ
Запускать когда хотите сохранить результаты
"""

print("="*60)
print("СКРИПТ 5: СОХРАНЕНИЕ РЕЗУЛЬТАТОВ")
print("="*60)

import os
import zipfile
from google.colab import drive, files
import glob

def save_to_google_drive():
    """Сохраняет результаты на Google Drive"""
    print("💾 СОХРАНЕНИЕ НА GOOGLE DRIVE...")

    # Подключаем Google Drive
    drive.mount('/content/drive')

    # Создаем папку для результатов
    import datetime
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    results_folder = f'/content/drive/MyDrive/LLM_Results_{timestamp}'

    os.makedirs(results_folder, exist_ok=True)

    # Копируем все CSV файлы
    csv_files = glob.glob('*.csv')

    if not csv_files:
        print("❌ Нет CSV файлов для сохранения")
        return

    saved_count = 0
    for csv_file in csv_files:
        try:
            # Копируем файл
            import shutil
            shutil.copy(csv_file, os.path.join(results_folder, csv_file))
            saved_count += 1
            print(f"   ✅ {csv_file}")
        except Exception as e:
            print(f"   ❌ Ошибка копирования {csv_file}: {e}")

    print(f"\n📁 Сохранено файлов: {saved_count}")
    print(f"📂 Папка на Google Drive: {results_folder}")

    # Отключаем Google Drive (опционально)
    drive.flush_and_unmount()

    return results_folder

def create_results_zip():
    """Создает ZIP архив со всеми результатами"""
    print("\n📦 СОЗДАНИЕ ZIP-АРХИВА...")

    csv_files = glob.glob('*.csv')

    if not csv_files:
        print("❌ Нет CSV файлов для архивации")
        return None

    zip_filename = 'ALL_RESULTS.zip'

    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file in csv_files:
            zipf.write(file)
            print(f"   📎 Добавлен: {file}")

    print(f"\n✅ Создан архив: {zip_filename}")
    print(f"   Размер: {os.path.getsize(zip_filename) / 1024:.1f} KB")

    return zip_filename

def download_results():
    """Скачивает результаты на компьютер"""
    print("\n📥 СКАЧИВАНИЕ РЕЗУЛЬТАТОВ...")

    # Создаем архив
    zip_file = create_results_zip()

    if zip_file and os.path.exists(zip_file):
        print("⏳ Начинается скачивание...")
        files.download(zip_file)
    else:
        print("❌ Не удалось создать архив для скачивания")

def show_results_info():
    """Показывает информацию о результатах"""
    print("\n📊 ИНФОРМАЦИЯ О РЕЗУЛЬТАТАХ:")

    csv_files = glob.glob('*.csv')
    prob_files = [f for f in csv_files if '_probabilities' in f]
    summary_files = [f for f in csv_files if 'SUMMARY' in f]

    print(f"📁 Всего CSV файлов: {len(csv_files)}")
    print(f"📄 Файлов с вероятностями: {len(prob_files)}")
    print(f"📋 Сводных отчетов: {len(summary_files)}")

    if prob_files:
        print("\n📈 ФАЙЛЫ С ВЕРОЯТНОСТЯМИ:")
        for file in sorted(prob_files):
            size_kb = os.path.getsize(file) / 1024
            print(f"   • {file:30} ({size_kb:.1f} KB)")

    if summary_files:
        print("\n📊 СВОДНЫЕ ОТЧЕТЫ:")
        for file in summary_files:
            try:
                df = pd.read_csv(file)
                print(f"   • {file:30} ({len(df)} строк)")
            except:
                print(f"   • {file:30} (ошибка чтения)")

# Меню выбора
print("\n🎯 ВЫБЕРИТЕ ДЕЙСТВИЕ:")
print("1. 📥 Скачать все результаты на компьютер")
print("2. 💾 Сохранить на Google Drive")
print("3. 📊 Показать информацию о результатах")
print("4. 🚫 Выход")

try:
    choice = input("\nВведите номер (1-4): ").strip()

    if choice == '1':
        download_results()
    elif choice == '2':
        save_to_google_drive()
    elif choice == '3':
        show_results_info()
    elif choice == '4':
        print("👋 Выход...")
    else:
        print("❌ Неверный выбор")

except Exception as e:
    print(f"❌ Ошибка: {e}")

СКРИПТ 5: СОХРАНЕНИЕ РЕЗУЛЬТАТОВ

🎯 ВЫБЕРИТЕ ДЕЙСТВИЕ:
1. 📥 Скачать все результаты на компьютер
2. 💾 Сохранить на Google Drive
3. 📊 Показать информацию о результатах
4. 🚫 Выход

Введите номер (1-4): 1

📥 СКАЧИВАНИЕ РЕЗУЛЬТАТОВ...

📦 СОЗДАНИЕ ZIP-АРХИВА...
   📎 Добавлен: Text_2_probabilities.csv
   📎 Добавлен: Text_1_probabilities.csv
   📎 Добавлен: ALL_TEXTS_SUMMARY.csv

✅ Создан архив: ALL_RESULTS.zip
   Размер: 23.1 KB
⏳ Начинается скачивание...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>